In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#funkcja dzieli dane na na zbiór treningowy i testowy
from sklearn.model_selection import train_test_split

#StandardScaler - skaluje dane numeryczne
#OneHotEncoder - zmienia kategorie na liczby
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#ColumnTransformer - pozwala stosować różne przekształcenia dla różnych kolumn
from sklearn.compose import ColumnTransformer

#pipeline - ciąg operacji przetwarzania danych i trenowania modelu połączonych w 1 obiekt, które można traktować jako pojedynczy model
#dane wchodzą i przechodzą przez etapy: czyszczenie (braki), skalowanie, kodowanie kategorii, trenowanie modelu
from sklearn.pipeline import Pipeline

#SimpleImputer - uzupełnia brakujące wartości
from sklearn.impute import SimpleImputer

#klasyfiaktor to model, który uczy się przypisywać dane wejściowe do jednej z określonych klas
#RandomForestClassifier - klasyfikator zbudowany z wielu drzew decyzyjnych
from sklearn.ensemble import RandomForestClassifier

#metryki do oceny modelu
from sklearn.metrics import classification_report, accuracy_score

In [5]:
#wczytuję dane z pliku csv i wyświetlam 5 pierwszych wierszy
data = pd.read_csv('data/obesity_data.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/obesity_data.csv'

In [ ]:
#info - pokazuje strukturę danych - liczbę wiersz, kolumnt itd
#describe - statystyki opisowe - średnia, min, max itd
print(data.info())          
print(data.describe(include='all')) 

In [ ]:
#wybieram interesujące mnie kolumny
selected_cols = [
    'Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
    'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC',
    'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad'
]
data = data[selected_cols]

In [ ]:
#wyświetlam ile wartości brakuje w każdej kolumnie
print("Missing values per column:\n", data.isnull().sum())

In [ ]:
#oddzielenie cech (x) i etykiet (y)
#x to dane wejściowe, czyli wszystko poza NObeyesdad
#y to dane wyjściowe, czyli NObeyesdad
X = data.drop("NObeyesdad", axis=1)
y = data["NObeyesdad"]

In [ ]:
#dzielimy kolumny na numeryczne i kategoryczne
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

In [ ]:
#robię mini pipelina dla kolumn liczbowych, jeśli jakiś danych brakuje, wstawiam średnią
#robię standaryzację, żeby średnia wynosiła 0, a odchylenie 1, aby algorytm działał lepiej
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [ ]:
#mini pipeline dla kolumn tekstowych - brakujące wartości zastępuje najczęściej występującą
#zmieniam na liczby - każda unikalna wartość staje się osobną kolumną zawierającą 0 lub 1
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
#łączę kolumny numeryczne i kategoryczne razem
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
#główny pipeline
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [ ]:
#dzielę dane na treningowe i testowe (80% na test)
#ustawiam seed, żeby wynik był powtarzalny
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#trenowanie modelu
clf.fit(X_train, y_train)

In [ ]:
#testowanie i ocena modelu
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))